# Analysis of Neighborhoods in Toronto

## By Ben Schloss

This notebook is dedicated to the assignment for week 3 of the Applied Data Science Capstone Course, the 4th and final course in IBM's Python for Data Science Specialization. In this notebook, I first import and process data on Toronto's neighborhoods by postal code. I then use Foursquare API to retrieve information on the types of venues in each neighborhood in Old Toronto in order compile a profile on each neighborhood. Finally, I use this data to run K-means clustering on the neighborhoods in order to determine which nieghborhoods are most similar and which are most distinct.



## Note: All 3 parts of assignment are submitted within this notebook. I inserted markdown (like this) to show where the answers to each part are! Thanks!

In [603]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium

from sklearn.cluster import KMeans

print('All libraries imported!')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

All libraries imported!


In [802]:
aa = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [803]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0].rename(columns={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'})
df = df.drop(df.index[df['Borough'] == 'Not assigned'])
df = df.reset_index()
df = df.drop(labels='index', axis=1)
df['Neighborhood'] = df['Neighborhood'].replace('Not assigned', df['Borough'])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


## FINAL TABLE FOR PART 1 BELOW:

In [804]:
df2 = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)
df2 = pd.DataFrame(data=df2)
df2['Neighborhood'] = df2['Neighborhood'].str.join(', ')
df2 = df2.reset_index()
df2.head(15)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## FINAL TABLE FOR PART 1 ABOVE!

In [805]:
bb = pd.read_csv('https://cocl.us/Geospatial_data')
bb.head(15)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


## ANSWER TO PART 2 HERE:

In [806]:
df2['Latitude'] = bb['Latitude']
df2['Longitude'] = bb['Longitude']
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [807]:
df2 = df2.set_index('PostalCode')
df2 = df2[['Borough', 'Latitude', 'Longitude', 'Neighborhood']]
df2.head(15)

,Borough,Latitude,Longitude,Neighborhood
PostalCode,,,,
M1B,Scarborough,43.806686,-79.194353,"Rouge, Malvern"
M1C,Scarborough,43.784535,-79.160497,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,43.763573,-79.188711,"Guildwood, Morningside, West Hill"
M1G,Scarborough,43.770992,-79.216917,Woburn
M1H,Scarborough,43.773136,-79.239476,Cedarbrae
M1J,Scarborough,43.744734,-79.239476,Scarborough Village
M1K,Scarborough,43.727929,-79.262029,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,43.711112,-79.284577,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,43.716316,-79.239476,"Cliffcrest, Cliffside, Scarborough Village West"


## Now, I will use FourSquare API to search venues near the M5R postal code in Central Toronto, just as a test to see the data before we get down to business!

## Answer to Part 3 (from here until end of notebook):

In [808]:
CLIENT_ID = '2E3HE34ZZO3QIDOW4MKQKDRGFO30NV1AGQE5OBY4ZUABRMRE'
CLIENT_SECRET = 'IJKYWZKEETG2ISNIGVRPNU13XX3K2FHCBGVU3V3Y1O0PB1XG'
VERSION = '20180604'
LIMIT = 50
radius = 600
latitude = df2.loc['M5R', 'Latitude']
longitude = df2.loc['M5R', 'Longitude']

In [809]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [810]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e3489f2949393001b9cb55b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Annex',
  'headerFullLocation': 'The Annex, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 33,
  'suggestedBounds': {'ne': {'lat': 43.678109705400004,
    'lng': -79.39822651036465},
   'sw': {'lat': 43.66730969459999, 'lng': -79.41313028963538}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c62c59ce1621b8dd0332453',
       'name': 'Roti Cuisine of India',
       'location': {'address': '308 Dupont St.',
        'crossStreet': 'at Walmer Rd.',
        'lat': 43.67461834990478,
        'lng': -79.40824866273744,
        'labeledLatLngs'

## Now, to real business. I am going to create a function that will allow me to use FourSquare API to scan multiple neighborhoods for venues and input the data into a dataframe. I will use the function to map the most common venues in each neighborhood of Old Toronto (by postal code)!

In [811]:
LIMIT = 30
def check_venues(names, latitudes, longitudes, radius=500):
    
    venues_list = []
    for name, lat, long in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat,
            long,
            radius,
            LIMIT)
    
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [628]:
ot_df = df2[df2['Borough'].str.contains('Toronto')].reset_index()
ot_df

,PostalCode,Borough,Latitude,Longitude,Neighborhood
0,M4E,East Toronto,43.676357,-79.293031,The Beaches
1,M4K,East Toronto,43.679557,-79.352188,"The Danforth West, Riverdale"
2,M4L,East Toronto,43.668999,-79.315572,"The Beaches West, India Bazaar"
3,M4M,East Toronto,43.659526,-79.340923,Studio District
4,M4N,Central Toronto,43.728020,-79.388790,Lawrence Park
5,M4P,Central Toronto,43.712751,-79.390197,Davisville North
6,M4R,Central Toronto,43.715383,-79.405678,North Toronto West
7,M4S,Central Toronto,43.704324,-79.388790,Davisville
8,M4T,Central Toronto,43.689574,-79.383160,"Moore Park, Summerhill East"
9,M4V,Central Toronto,43.686412,-79.400049,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."


In [812]:
old_toronto_venues = check_venues(names=ot_df['PostalCode'], latitudes=ot_df['Latitude'], longitudes=ot_df['Longitude'])
old_toronto_venues.head(20)

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7A
M7Y


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,M4K,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
6,M4K,43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
7,M4K,43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
8,M4K,43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
9,M4K,43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop


In [813]:
old_toronto_cat1 = pd.get_dummies(old_toronto_venues[['Venue Category']], prefix = '', prefix_sep= '')
old_toronto_cat1 = old_toronto_cat1.drop('Neighborhood', axis=1)
old_toronto_cat1.insert(loc=0, column='Neighborhood', value=old_toronto_venues['Neighborhood'],)

old_toronto_cat1.head(20)

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [814]:
old_toronto_grouped = old_toronto_cat1.groupby('Neighborhood', axis=0).mean().reset_index()
old_toronto_grouped.head(20)

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25000

In [815]:
num_top_venues = 5

for hood in old_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = old_toronto_grouped[old_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0  Health Food Store  0.25
1                Pub  0.25
2              Trail  0.25
3            Airport  0.00
4             Museum  0.00


----M4K----
                venue  freq
0    Greek Restaurant  0.30
1  Italian Restaurant  0.07
2      Ice Cream Shop  0.07
3         Yoga Studio  0.03
4     Bubble Tea Shop  0.03


----M4L----
              venue  freq
0              Park  0.10
1    Sandwich Place  0.10
2               Gym  0.05
3  Sushi Restaurant  0.05
4               Pub  0.05


----M4M----
                 venue  freq
0                 Café  0.13
1          Coffee Shop  0.10
2  American Restaurant  0.07
3   Italian Restaurant  0.07
4               Bakery  0.07


----M4N----
           venue  freq
0           Park  0.33
1       Bus Line  0.33
2    Swim School  0.33
3        Airport  0.00
4  Movie Theater  0.00


----M4P----
               venue  freq
0                Gym  0.14
1     Breakfast Spot  0.14
2               Park  0.14
3  Food & Dr

In [816]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [817]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = old_toronto_grouped['Neighborhood']

for ind in np.arange(old_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:-1] = return_most_common_venues(old_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood
0,NaN,Trail,Pub,Health Food Store,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,M4E
1,NaN,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Juice Bar,Bookstore,Restaurant,Spa,Diner,M4K
2,NaN,Sandwich Place,Park,Pet Store,Burger Joint,Burrito Place,Italian Restaurant,Ice Cream Shop,Liquor Store,Pub,Fast Food Restaurant,M4L
3,NaN,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Middle Eastern Restaurant,Coworking Space,Comfort Food Restaurant,Ice Cream Shop,Stationery Store,M4M
4,NaN,Park,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,M4N


In [818]:
neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop(neighborhoods_venues_sorted.columns[0], axis=1)
cols = list(neighborhoods_venues_sorted)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
neighborhoods_venues_sorted = neighborhoods_venues_sorted.loc[:, cols]
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Trail,Pub,Health Food Store,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,M4K,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Juice Bar,Bookstore,Restaurant,Spa,Diner
2,M4L,Sandwich Place,Park,Pet Store,Burger Joint,Burrito Place,Italian Restaurant,Ice Cream Shop,Liquor Store,Pub,Fast Food Restaurant
3,M4M,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Middle Eastern Restaurant,Coworking Space,Comfort Food Restaurant,Ice Cream Shop,Stationery Store
4,M4N,Park,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


In [819]:
bb = bb.rename(columns={'Postal Code':'Neighborhood'})
df_merged = bb.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged.head(30)

,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,43.806686,-79.194353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,43.784535,-79.160497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,43.763573,-79.188711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M1G,43.770992,-79.216917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,43.773136,-79.239476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M1J,43.744734,-79.239476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1K,43.727929,-79.262029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,M1L,43.711112,-79.284577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,M1M,43.716316,-79.239476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,M1N,43.692657,-79.264848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [820]:
df_merged = df_merged.dropna(axis=0)
df_merged.head()

,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,43.676357,-79.293031,Trail,Pub,Health Food Store,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
41,M4K,43.679557,-79.352188,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Juice Bar,Bookstore,Restaurant,Spa,Diner
42,M4L,43.668999,-79.315572,Sandwich Place,Park,Pet Store,Burger Joint,Burrito Place,Italian Restaurant,Ice Cream Shop,Liquor Store,Pub,Fast Food Restaurant
43,M4M,43.659526,-79.340923,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Middle Eastern Restaurant,Coworking Space,Comfort Food Restaurant,Ice Cream Shop,Stationery Store
44,M4N,43.728020,-79.388790,Park,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


## Finally, I will use K-means clustering to group the neighborhoods by similarity in terms of venues. Having run the analysis multiple times, I determined that 3 clusters was most appropriate given the data. As you will be able to see, the first cluster of neighborhoods is comprised of mostly restauraunt dominant neighborhoods in central and west-central downtown Toronto, while the second cluster is comprised mainly of recreation oriented neighborhoods closer to North/Midtown Toronto.

In [821]:
kclusters = 3

toronto_group_clust = old_toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_group_clust)

kmeans.labels_[0:10] 

array([1, 0, 0, 0, 1, 1, 0, 0, 1, 0], dtype=int32)

In [782]:
df_merged.insert(0,'Cluster Labels', kmeans.labels_)
df_merged

,Cluster Labels,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,1,M4E,43.676357,-79.293031,Trail,Pub,Health Food Store,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
41,0,M4K,43.679557,-79.352188,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Juice Bar,Bookstore,Restaurant,Spa,Diner
42,0,M4L,43.668999,-79.315572,Sandwich Place,Park,Pet Store,Burger Joint,Burrito Place,Italian Restaurant,Ice Cream Shop,Liquor Store,Pub,Fast Food Restaurant
43,0,M4M,43.659526,-79.340923,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Middle Eastern Restaurant,Coworking Space,Comfort Food Restaurant,Ice Cream Shop,Stationery Store
44,1,M4N,43.728020,-79.388790,Park,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
45,1,M4P,43.712751,-79.390197,Hotel,Park,Breakfast Spot,Gym,Sandwich Place,Food & Drink Shop,Department Store,Yoga Studio,Eastern European Restaurant,Dumpling Restaurant
46,0,M4R,43.715383,-79.405678,Clothing Store,Coffee Shop,Sporting Goods Shop,Gym / Fitness Center,Health & Beauty Service,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park
47,0,M4S,43.704324,-79.388790,Dessert Shop,Italian Restaurant,Gym,Pizza Place,Café,Sandwich Place,Sushi Restaurant,Coffee Shop,Toy / Game Store,Farmers Market
48,1,M4T,43.689574,-79.383160,Gym,Trail,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
49,0,M4V,43.686412,-79.400049,Pub,Coffee Shop,Restaurant,Sushi Restaurant,Liquor Store,Sports Bar,Light Rail Station,Fried Chicken Joint,Supermarket,American Restaurant


In [783]:
import matplotlib.cm as cm
import matplotlib.colors as colors


In [784]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow, #[cluster-1]
        fill=True,
        fill_color=rainbow, #[cluster-1]
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [791]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[list(range(0, df_merged.shape[1]))]]

,Cluster Labels,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,0,M4K,43.679557,-79.352188,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Juice Bar,Bookstore,Restaurant,Spa,Diner
42,0,M4L,43.668999,-79.315572,Sandwich Place,Park,Pet Store,Burger Joint,Burrito Place,Italian Restaurant,Ice Cream Shop,Liquor Store,Pub,Fast Food Restaurant
43,0,M4M,43.659526,-79.340923,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Middle Eastern Restaurant,Coworking Space,Comfort Food Restaurant,Ice Cream Shop,Stationery Store
46,0,M4R,43.715383,-79.405678,Clothing Store,Coffee Shop,Sporting Goods Shop,Gym / Fitness Center,Health & Beauty Service,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park
47,0,M4S,43.704324,-79.388790,Dessert Shop,Italian Restaurant,Gym,Pizza Place,Café,Sandwich Place,Sushi Restaurant,Coffee Shop,Toy / Game Store,Farmers Market
49,0,M4V,43.686412,-79.400049,Pub,Coffee Shop,Restaurant,Sushi Restaurant,Liquor Store,Sports Bar,Light Rail Station,Fried Chicken Joint,Supermarket,American Restaurant
51,0,M4X,43.667967,-79.367675,Coffee Shop,Italian Restaurant,Restaurant,Café,Park,Pub,Sandwich Place,Japanese Restaurant,Jewelry Store,Butcher
52,0,M4Y,43.665860,-79.383160,Gay Bar,Burger Joint,Park,Pub,Indian Restaurant,Ethiopian Restaurant,Breakfast Spot,Salon / Barbershop,Bubble Tea Shop,Restaurant
53,0,M5A,43.654260,-79.360636,Coffee Shop,Park,Café,Pub,Mexican Restaurant,Bakery,Ice Cream Shop,Farmers Market,Event Space,Dessert Shop
54,0,M5B,43.657162,-79.378937,Coffee Shop,Café,Clothing Store,Movie Theater,Burrito Place,Plaza,Spa,Electronics Store,Hotel,Burger Joint


In [793]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[list(range(0, df_merged.shape[1]))]]

,Cluster Labels,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,1,M4E,43.676357,-79.293031,Trail,Pub,Health Food Store,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
44,1,M4N,43.728020,-79.388790,Park,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
45,1,M4P,43.712751,-79.390197,Hotel,Park,Breakfast Spot,Gym,Sandwich Place,Food & Drink Shop,Department Store,Yoga Studio,Eastern European Restaurant,Dumpling Restaurant
48,1,M4T,43.689574,-79.383160,Gym,Trail,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
50,1,M4W,43.679563,-79.377529,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
64,1,M5P,43.696948,-79.411307,Trail,Sushi Restaurant,Mexican Restaurant,Jewelry Store,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [794]:
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[list(range(0, df_merged.shape[1]))]]

,Cluster Labels,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,2,M5N,43.711695,-79.416936,Pool,Garden,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
